In [1]:
pip install tweepy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tweepy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import pandas as pd
import nltk


In [3]:
import time

In [4]:
# Download NLTK resources (only needed the first time)
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAAilyAEAAAAAsTUHC07DT6P5DXAQPtXXFTHFqlE%3Dq9JxaCVUulyX8QeyNLBFd83l3ZuNeGWX7Iy2SfTZgZUzcIH8f8"

In [6]:

# Step 1: Fetch Tweets
def fetch_tweets(topic, count=10):
    client = tweepy.Client(bearer_token=BEARER_TOKEN)
    try:
        response = client.search_recent_tweets(query=topic, max_results=count, tweet_fields=["created_at"])
        tweets = [{"text": tweet.text, "created_at": tweet.created_at} for tweet in response.data]
        return tweets
    except Exception as e:
        print(f"Error fetching tweets: {e}")
        return []

In [7]:
def handle_rate_limits(client, query, max_results):
    try:
        response = client.search_recent_tweets(query=query, max_results=max_results)
        return response.data
    except tweepy.errors.TooManyRequests as e:
        reset_time = int(e.response.headers.get("x-rate-limit-reset", time.time() + 15 * 60))
        wait_time = reset_time - int(time.time())
        print(f"Rate limit reached. Waiting {wait_time} seconds.")
        time.sleep(wait_time)
        return handle_rate_limits(client, query, max_results)



In [8]:
def fetch_paginated_tweets(client, query, max_results, max_pages=3):
    tweets = []
    next_token = None

    for _ in range(max_pages):
        response = client.search_recent_tweets(query=query, max_results=max_results, next_token=next_token)
        if response.data:
            tweets.extend(response.data)
        next_token = response.meta.get("next_token")
        if not next_token:
            break  # No more pages
    return tweets

In [9]:
# Step 2: Analyze Sentiment
def analyze_sentiment(tweets):
    sid = SentimentIntensityAnalyzer()
    for tweet in tweets:
        sentiment_score = sid.polarity_scores(tweet["text"])["compound"]
        tweet["sentiment"] = sentiment_score
    return tweets

In [10]:

# Step 3: Visualize Sentiment
def visualize_sentiment(tweets):
    df = pd.DataFrame(tweets)
    df["created_at"] = pd.to_datetime(df["created_at"])
    df = df.set_index("created_at").resample("H").mean()  # Resample sentiment by hour
    df["sentiment"].plot(kind="line", figsize=(10, 6), title="Sentiment Trend Over Time", ylabel="Sentiment Score")
    plt.axhline(0, color='red', linestyle='--')  # Neutral sentiment line
    plt.show()


In [11]:
# Main Function
if __name__ == "__main__":
    topic = "narendra modi"  # Replace with your topic of interest
    print(f"Fetching tweets about '{topic}'...")
    
    tweets = fetch_tweets(topic, count=50)  # Fetch tweets
    if tweets:
        tweets = analyze_sentiment(tweets)  # Analyze sentiment
        visualize_sentiment(tweets)  # Visualize sentiment
    else:
        print("No tweets fetched.")

Fetching tweets about 'narendra modi'...
Error fetching tweets: 429 Too Many Requests
Usage cap exceeded: Monthly product cap
No tweets fetched.
